## Inicializacion

In [1]:
!pip install gdown
!pip install deep-translator
!pip install imbalanced-learn
!pip install matplotlib
!pip install seaborn
!pip install scikit-learn
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.6 MB/s eta 0:00:00


In [36]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from deep_translator import GoogleTranslator
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
import string
import re
import os
import numpy as np
import warnings
import gdown
import ipywidgets as widgets
from IPython.display import display
import time
import sys

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.stop_words = set(stopwords.words('english'))  # Cambia a 'spanish' si trabajas con frases en español

    def clean_text(self, text):
        # Convertir a minúsculas
        text = text.lower()
        # Eliminar puntuación
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Eliminar stopwords
        text = ' '.join([word for word in text.split() if word not in self.stop_words])
        return text

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if isinstance(X, pd.Series):
            return X.apply(self.clean_text)
        elif isinstance(X, str):
            return self.clean_text(X)
        else:
            raise ValueError("Input must be a string or pandas Series.")

In [21]:
def load_database():
    file_id = '1xkN6-OKnp8XOCjcRXFHyFAmKN6GODpji'
    download_url = f'https://drive.google.com/uc?id={file_id}'
    output = 'database/bgg_database.csv'
    gdown.download(download_url, output, quiet=True)


    file_id = '1b4PUV-SRkUm7A_vLeRZMkA80rsABJjuV'
    download_url = f'https://drive.google.com/uc?id={file_id}'
    output = 'database/IMDB-Movie-Data.csv'
    gdown.download(download_url, output, quiet=True)

    file_id = '1zJYm3kKzy1HQzta6aCmikTENgPOIrx28'
    download_url = f'https://drive.google.com/uc?id={file_id}'
    output = 'database/libros.csv'
    gdown.download(download_url, output, quiet=True)

    file_id = '1dfduFDeHbIFoXhNj8FCV7yvuvwKn_rpl'
    download_url = f'https://drive.google.com/uc?id={file_id}'
    output = 'database/negative-words.txt'
    gdown.download(download_url, output, quiet=True)

    file_id = '17jI37fqKDp9yzgAiwTIE3UmFvkL5XHO1'
    download_url = f'https://drive.google.com/uc?id={file_id}'
    output = 'database/positive-words.txt'
    gdown.download(download_url, output, quiet=True)


## Carga

In [5]:
def load_books(developer_mode):
    # URL de la página principal
    url = "https://www.gutenberg.org/browse/scores/top1000.php#books-last1"

    # Hacer la solicitud a la página principal
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # Seleccionar solo los enlaces de libros que contienen '/ebooks/' seguido de un número
    libros = soup.select("li > a[href^='/ebooks/']")
    datos_libros = []

    # Extraer datos de cada libro
    for libro in libros:
        texto_completo = libro.get_text()
        enlace_anidado = libro['href']

        # Ignorar enlaces que no tienen números después de "/ebooks/"
        if enlace_anidado.split('/ebooks/')[-1].isdigit():
            titulo_y_autor = texto_completo.split(" by ", 1)
            titulo = titulo_y_autor[0].strip()
            autor = titulo_y_autor[1].strip() if len(titulo_y_autor) > 1 else "Desconocido"

            # URL completa del libro
            url_libro = f"https://www.gutenberg.org{enlace_anidado}"

            # Hacer la solicitud a la página del libro
            response_libro = requests.get(url_libro)
            response_libro.raise_for_status()
            soup_libro = BeautifulSoup(response_libro.text, 'html.parser')

            # Intentar extraer el summary
            summary = ""
            summary_row = soup_libro.find('th', text='Summary')
            if summary_row:
                summary_td = summary_row.find_next("td")
                if summary_td:
                    summary = summary_td.get_text(strip=True)

            # Extraer los subjects
            subjects = []
            subject_rows = soup_libro.find_all('th', text='Subject')
            for subject_row in subject_rows:
                subject_td = subject_row.find_next("td")
                if subject_td:
                    subject_links = subject_td.find_all('a')
                    for link in subject_links:
                        subjects.append(link.get_text(strip=True))

            # Unir todos los subjects en una sola cadena
            subjects_combined = ", ".join(subjects)

            # Agregar datos al listado
            datos_libros.append({
                "Título": titulo,
                "Autor": autor,
                "Enlace": url_libro,
                "Resumen": summary,
                "Subjects": subjects_combined
            })
            if developer_mode:
                print("titulo:", titulo, "autor:", autor, "url:", url_libro, "resumen:", summary, "subjects:", subjects_combined)

    # Crear el DataFrame
    df_libros = pd.DataFrame(datos_libros)
    df_libros.to_csv('database/libros.csv', index=False)

In [26]:
# Función para cargar palabras desde un archivo
def load_words(file_path):
    with open(file_path, 'r', encoding='latin-1') as file:  # Cambia a 'utf-16' si es necesario
        words = [line.strip() for line in file.readlines()]
    return set(words)

In [7]:
def load_datasets():
    # Instanciar el preprocesador de texto
    text_preprocessor = TextPreprocessor()

    # 1. Películas
    peliculas_dataframe = pd.read_csv('database/IMDB-Movie-Data.csv')
    peliculas_dataframe['category'] = 'pelicula'
    peliculas_dataframe['text'] = (
        peliculas_dataframe['Title'].fillna('').apply(text_preprocessor.transform) + " " +
        peliculas_dataframe['Genre'].fillna('').apply(text_preprocessor.transform) + " " +
        peliculas_dataframe['Description'].fillna('').apply(text_preprocessor.transform) + " " +
        peliculas_dataframe['Director'].fillna('').apply(text_preprocessor.transform) + " " +
        peliculas_dataframe['Actors'].fillna('').apply(text_preprocessor.transform)
    )
    peliculas_dataframe.rename(columns={'Title': 'Titulo', 'Director': 'Autor', 'Description': 'Resumen', 'Genre': 'Subjects'}, inplace=True)

    # 2. Juegos de Mesa
    juegos_mesa_dataframe = pd.read_csv('database/bgg_database.csv')
    juegos_mesa_dataframe['category'] = 'juego'
    juegos_mesa_dataframe['text'] = (
        juegos_mesa_dataframe['game_name'].fillna('').apply(text_preprocessor.transform) + " " +
        juegos_mesa_dataframe['description'].fillna('').apply(text_preprocessor.transform) + " " +
        juegos_mesa_dataframe['designers'].apply(lambda x: ' '.join(eval(x)) if isinstance(x, str) else '').apply(text_preprocessor.transform).fillna('') + " " +
        juegos_mesa_dataframe['mechanics'].apply(lambda x: ' '.join(eval(x)) if isinstance(x, str) else '').apply(text_preprocessor.transform).fillna('') + " " +
        juegos_mesa_dataframe['categories'].apply(lambda x: ' '.join(eval(x)) if isinstance(x, str) else '').apply(text_preprocessor.transform).fillna('')
    )
    juegos_mesa_dataframe.rename(columns={'game_name': 'Titulo','game_href': 'Enlace', 'designers': 'Autor', 'description': 'Resumen', 'mechanics': 'Subjects'}, inplace=True)

    # 3. Libros
    libros_dataframe = pd.read_csv('database/libros.csv')
    libros_dataframe['category'] = 'libro'
    libros_dataframe['text'] = (
        libros_dataframe['Título'].fillna('').apply(text_preprocessor.transform) + " " +
        libros_dataframe['Resumen'].fillna('').apply(text_preprocessor.transform) + " " +
        libros_dataframe['Subjects'].fillna('').apply(text_preprocessor.transform) + " " +
        libros_dataframe['Autor'].fillna('').apply(text_preprocessor.transform)
    )
    libros_dataframe.rename(columns={'Título': 'Titulo', 'Autor': 'Autor', 'Resumen': 'Resumen', 'Subjects': 'Subjects'}, inplace=True)

    # Concatenar los DataFrames manteniendo el original y agregando las nuevas columnas
    dataframe_bbdd = pd.concat(
        [peliculas_dataframe[['Titulo', 'Autor', 'Resumen', 'Subjects', 'category', 'text']],
         juegos_mesa_dataframe[['Titulo', 'Enlace','Autor', 'Resumen', 'Subjects', 'category', 'text']],
         libros_dataframe[['Titulo', 'Enlace','Autor', 'Resumen', 'Subjects', 'category', 'text']]
        ],
        ignore_index=True
    )

    return dataframe_bbdd

In [42]:
def print_slow(text, delay=0.04):
    for char in text:
        sys.stdout.write(char)
        sys.stdout.flush()  # Asegura que el texto se imprima inmediatamente
        time.sleep(delay)   # Pausa entre cada carácter
    print()  # Para agregar una nueva línea al final

## Modelos

In [8]:
# Función para contar palabras
def count_words(text, positive_words, negative_words):
    positive_count = sum(1 for word in text.split() if word in positive_words)
    negative_count = sum(1 for word in text.split() if word in negative_words)
    return pd.Series([positive_count, negative_count])

# Función para preprocesar la entrada del usuario
def procesar_entrada_usuario(texto):
    # Traducir al inglés
    texto_traducido = GoogleTranslator(source='es', target='en').translate(texto)
    # Preprocesar el texto traducido
    texto_procesado = TextPreprocessor().clean_text(texto_traducido)
    return texto_procesado


In [9]:

def model_sentiments(developer_mode):    # Cargar palabras positivas y negativas
    positive_words = load_words('database/positive-words.txt')
    negative_words = load_words('database/negative-words.txt')# Crear un conjunto de datos basado en las palabras
    data = {
        'text': [],
        'label': []
    }

    # Llenar el conjunto de datos con palabras positivas y negativas
    for word in positive_words:
        data['text'].append(word)
        data['label'].append('feliz')

    for word in negative_words:
        data['text'].append(word)
        data['label'].append('triste')

    # Crear un DataFrame
    df = pd.DataFrame(data)

    # Crear características
    df[['positive_count', 'negative_count']] = df['text'].apply(lambda x: count_words(x, positive_words, negative_words))

    # Convertir etiquetas a números
    df['label'] = df['label'].map({'feliz': 1, 'triste': 0})

    # Separar características y etiquetas
    X = df[['positive_count', 'negative_count']]
    y = df['label']

    # Dividir el conjunto de datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

    # Entrenar el modelo de regresión logística
    model = LogisticRegression()
    model.fit(X_train, y_train)

    if developer_mode:
        # Predecir en el conjunto de prueba
        y_pred = model.predict(X_test)

        # Evaluar el modelo
        accuracy = accuracy_score(y_test, y_pred)
        confusion = confusion_matrix(y_test, y_pred)
        report = classification_report(y_test, y_pred)

        print(f'Accuracy: {accuracy}')
        print('Confusion Matrix:')
        print(confusion)
        print('Classification Report:')
        print(report)

    return model, positive_words, negative_words

In [10]:
def train_model(dataset, developer_mode):
    # Separar los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(dataset['text'], dataset['category'], test_size=0.2, random_state=42)

    # Crear un pipeline con TF-IDF y regresión logística con hiperparámetros
    model = make_pipeline(
        TfidfVectorizer(),
        LogisticRegression(C=1.0, max_iter=200, solver='liblinear')
    )

    # Entrenar el modelo
    model.fit(X_train, y_train)

    # Validación cruzada
    cv_scores = cross_val_score(model, X_train, y_train, cv=5)  # 5-fold cross-validation
    if developer_mode:
        print(f"Cross-validation scores: {cv_scores}")
        print(f"Mean cross-validation score: {cv_scores.mean()}")

    return model

In [11]:
def recommend_item(prompt, model):
    # Predecir la categoría del prompt
    category = model.predict([prompt])[0]
    return category

In [12]:
def model_recommender(dataset):
    # Vectorizar el texto combinado
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(dataset['text'])

    return tfidf_matrix, tfidf_vectorizer

In [13]:
def predict_sentiments(model, positive_words, negative_words, frase, developer_mode):

    entrada_procesada = procesar_entrada_usuario(frase)
    if developer_mode:
        print("Texto procesado:", entrada_procesada)

    # Contar palabras en la entrada procesada
    entrada_counts = count_words(entrada_procesada, positive_words, negative_words)

    # Asegúrate de que la entrada esté en formato 2D
    entrada_vectorizada = np.array([entrada_counts]).reshape(1, -1)

    # Obtener la predicción de la clase más probable
    prediccion = model.predict(entrada_vectorizada)[0]

    # Determinar el nombre de la emoción
    emocion = 'feliz' if prediccion == 1 else 'triste'

    return emocion

In [39]:
def input_user():
    prompt_message = "¿Cómo estás hoy? ¿Qué quieres hacer? Puedes despedirte diciendo 'Chau': \n"
    print_slow(prompt_message)  # Imprimir el mensaje lentamente
    prompt = input()  # Leer la entrada del usuario
    if prompt.lower() == "chau":
        return "exit", "exit"

    # Separar el prompt en sentimientos y recomendaciones por la coma
    prompts = prompt.split(",")

    # Limpiar y asignar los textos
    sentiment_prompt = prompts[0].strip()  # Texto antes de la coma
    recommendation_prompt = prompts[1].strip() if len(prompts) > 1 else ""  # Texto después de la coma

    # Traducir los prompts
    translated_sentiment = GoogleTranslator(source='es', target='en').translate(sentiment_prompt)
    translated_recommendation = GoogleTranslator(source='es', target='en').translate(recommendation_prompt)

    # Procesar el texto
    prompt_sentimiento = TextPreprocessor().clean_text(translated_sentiment)
    prompt_recomendacion = TextPreprocessor().clean_text(translated_recommendation)

    return prompt_sentimiento, prompt_recomendacion

In [35]:
def predict_recomendation(prompt, category, model, dataframe_bbdd):
    # Filtrar el dataframe_bbdd para obtener solo las recomendaciones de la categoría dada
    recommendations = dataframe_bbdd[dataframe_bbdd['category'] == category].copy().reset_index(drop=True)

    if recommendations.empty:
        return "No hay recomendaciones disponibles", ""

    # Vectorizar el prompt de recomendación
    prompt_vector = model.named_steps['tfidfvectorizer'].transform([prompt])

    # Vectorizar el texto del dataset
    dataset_vector = model.named_steps['tfidfvectorizer'].transform(recommendations['text'])

    # Calcular la similitud de coseno entre el prompt y el texto del dataset
    similarity_scores = cosine_similarity(prompt_vector, dataset_vector)

    # Obtener el índice del ítem más similar en el dataset
    most_similar_idx = similarity_scores.argmax()

    # Obtener la recomendación correspondiente del dataframe_bbdd
    recommended_item = recommendations.iloc[most_similar_idx]

    return recommended_item['Titulo'], recommended_item['Enlace']

In [40]:
def main(developer_mode):
    # Verificar si el archivo libros.csv existe
    if not os.path.exists('database/bgg_database.csv') or not os.path.exists('database/IMDB-Movie-Data.csv') or not os.path.exists('database/libros.csv') or not os.path.exists('positive-words.txt') or not os.path.exists('negative-words.txt'):
        if not os.path.exists('database'):
            os.makedirs('database')
        load_database()
    if not os.path.exists('database/libros.csv'):
        load_books()

    model_sents, positive_words, negative_words = model_sentiments(developer_mode)

    dataframe_bbdd = load_datasets()

    recommendation_model = train_model(dataframe_bbdd, developer_mode)  # Asegúrate de que este modelo esté entrenado

    while True:
        prompt_sentimiento, prompt_recomendacion = input_user()

        if prompt_sentimiento == "exit":
            break

        sentimiento = predict_sentiments(model_sents, positive_words, negative_words, prompt_sentimiento, developer_mode)

        recommended_category = recommend_item(prompt_recomendacion, recommendation_model)  # Predecir la categoría

        if developer_mode:
            print("Prompt sentimiento:", prompt_sentimiento)
            print("Prompt recomendación:", prompt_recomendacion)
            print("category:", recommended_category)

        recommended_title, recommended_link = predict_recomendation(prompt_recomendacion, recommended_category, recommendation_model, dataframe_bbdd)  # Usar el prompt y el dataset
        print("")
        print_slow(f"Entiendo que hoy estas {sentimiento}")
        print_slow(f"Mi recomendación es: {recommended_title} y el enlace es: {recommended_link} \n")

Hoy fue un buen dia, quiero ver una pelicula de criminales intergalacticos
Tenemos un dia increible con amigos, queremos un juego de mesa de aventuras y ciencia ficcion
Hoy fue un dia horrible, quiero leer un libro de biologia
Hoy me encuentro mal, quiero ver una pelicula de comedia

## Bot


In [46]:
def chatbot():
    developer_mode = False
    warnings.filterwarnings("ignore")
    if input("¿Desea activar el modo desarrollador? (s/n): \n") == "s":
        developer_mode = True
    main(developer_mode)
    print("Nos vemos 👋")


## Chat

In [48]:
chatbot()

¿Desea activar el modo desarrollador? (s/n): 
n
¿Cómo estás hoy? ¿Qué quieres hacer? Puedes despedirte diciendo 'Chau': 

Hoy no estoy bien, no quiero una pelicula de comedia

Entiendo que hoy estas feliz
Mi recomendación es: Disaster Movie y el enlace es: nan 

¿Cómo estás hoy? ¿Qué quieres hacer? Puedes despedirte diciendo 'Chau': 

No

Entiendo que hoy estas triste
Mi recomendación es: Guardians of the Galaxy y el enlace es: nan 

¿Cómo estás hoy? ¿Qué quieres hacer? Puedes despedirte diciendo 'Chau': 

Chau
Nos vemos 👋


In [ ]:
pd.read_txt